#まとめ

* train (最初の 3 週間)
* valid （第4週の前半）
* valid label（第4週の後半）
* all train (train + valid + valid label ：最初の4週間)
* test (第5週の前半)
* LB (第5週の後半)

クリスディスカッション
https://www.kaggle.com/competitions/otto-recommender-system/discussion/370210

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/train_valid_validlabel/otto-train-and-test-data-for-local-validation/'
OUTPUT_DIR = f'/content/drive/MyDrive/kaggle/2022/OTTO/output/exp/{CFG.exp}/'

if not os.path.exists(OUTPUT_DIR):
  os.makedirs(OUTPUT_DIR)

#config

In [ ]:
class CFG:
  #実験ver
  exp = '1'

  #target type
  types = 'clicks' 

  #candidates
  candidates_count = 50

  #recall@k
  recall = 20 

  #step1ルール
  rules = 'covis'

  #CV
  cv_dataset = ''
  lb_dataset = ''

  #特徴量
  features = ['aa',
              'bbb',
              ]

In [ ]:
FEATURE_DIR = INPURT_DIR + CFG.features

In [ ]:
features = []

In [ ]:
import numpy as np
import pandas as pd

import os
import collections
from collections import Counter

import lightgbm as lgb
from sklearn.model_selection import GroupKFold
import pickle

import itertools
#import glob

import gc

# Data Load

In [ ]:
df_train = pd.read_parquet(INPUT_DIR+'train.parquet')
df_val = pd.read_parquet(INPUT_DIR+'test.parquet')

print(len(df_train))
print(len(df_val))

163955180
7683577


#Utils

In [ ]:
#メモリ削減関数

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
#特徴量読み込み関数


In [ ]:
#特徴量読み込み関数


In [ ]:
#debug

#df_train = df_train.head(100000)
#df_val = df_val.head(100000)
#print(len(df_train))
#print(len(df_val))

# stage1. candidate 生成

# step1 sessionのごとにitem候補生成 : valid

In [ ]:
%%time
# train & valid covis 読み込み

COVIS_MATRIX_PATH = '/content/drive/MyDrive/kaggle/2022/OTTO/input/train_covistation_matrix/chris_baseline/'

VER = 6
DISK_PIECES = 4

def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

# train & valid covis
# LOAD THREE CO-VISITATION MATRICES
top_20_clicks = pqt_to_dict( pd.read_parquet(COVIS_MATRIX_PATH+f'top_20_clicks_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(COVIS_MATRIX_PATH+f'top_20_clicks_v{VER}_{k}.pqt') ) )

CPU times: user 34.5 s, sys: 2.27 s, total: 36.8 s
Wall time: 43.9 s


In [ ]:
#人気
val_popular_aid = df_val.loc[df_val['type']==CFG.types,'aid'].value_counts().index.values[:20]

chris baselineルール

In [ ]:
# step1

type_weight_multipliers = {0: 1, 1: 6, 2: 3}


def generate_candidate(df, k=CFG.candidates_count):

  # USER HISTORY AIDS AND TYPES
  aids=df.aid.tolist()
  types = df.type.tolist()
  unique_aids = list(dict.fromkeys(aids[::-1] ))
  # RERANK CANDIDATES USING WEIGHTS
  if len(unique_aids)>=20:
      weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
      aids_temp = Counter() 
      # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
      for aid,w,t in zip(aids,weights,types): 
          aids_temp[aid] += w * type_weight_multipliers[t]
      sorted_aids = [k for k,v in aids_temp.most_common(k)]
      return sorted_aids[:k]

  # USE "CLICKS" CO-VISITATION MATRIX
  aids2 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
  # RERANK CANDIDATES
  top_aids2 = [aid2 for aid2, cnt in Counter(aids2).most_common(k) if aid2 not in unique_aids]    
  result = unique_aids + top_aids2[:k]
  # USE TOP20 TEST CLICKS
  return (result + list(val_popular_aid)[:k])[:k]

In [ ]:
from tqdm import tqdm
tqdm.pandas()

candidates = df_val.groupby('session').progress_apply(lambda x: generate_candidate(x)).explode()
candidates.name = 'item'
candidates = candidates.to_frame().reset_index()
candidates

100%|██████████| 1801251/1801251 [03:55<00:00, 7657.02it/s]


,session,item
0,11098528,11830
1,11098528,588923
2,11098528,1732105
3,11098528,571762
4,11098528,884502
...,...,...
56370876,12899778,13568
56370877,12899778,1097813
56370878,12899778,1102546
56370879,12899778,1068655


# step2 item特徴 : train + valid

In [ ]:
%%time
item_features = pd.merge(df_train, df_val, on=['session','aid','type'], how='left').groupby('aid').agg({'aid': 'count', 'session': 'nunique', 'type': 'mean'})
item_features.columns = ['item_item_count', 'item_user_count', 'item_by_count']
#item_features.to_parquet('item_features.pqt')
item_features

CPU times: user 2min 10s, sys: 9.05 s, total: 2min 19s
Wall time: 2min 18s


,item_item_count,item_user_count,item_by_count
aid,,,
0,33,26,0.000000
1,32,29,0.031250
2,12,12,0.000000
3,1201,668,0.079933
4,140,97,0.035714
...,...,...,...
1855598,6,6,0.000000
1855599,9,8,0.000000
1855600,68,42,0.073529


このアイテムがトレイン内でクリック,cart,orderされた回数

In [ ]:
%%time
item_click_count_features = df_train[df_train['type']==0].groupby('aid').agg({'session': 'count'})
item_click_count_features.columns = ['item_click_count_features']
#item_features.to_parquet('item_features.pqt')
item_click_count_features

item_cart_count_features = df_train[df_train['type']==1].groupby('aid').agg({'session': 'count'})
item_cart_count_features.columns = ['item_cart_count_features']
#item_features.to_parquet('item_features.pqt')
item_cart_count_features

item_order_count_features = df_train[df_train['type']==2].groupby('aid').agg({'session': 'count'})
item_order_count_features.columns = ['item_order_count_features']
#item_features.to_parquet('item_features.pqt')

item_features = item_features.merge(item_click_count_features, on=['aid'], how='outer').fillna(0)
item_features = item_features.merge(item_cart_count_features, on=['aid'], how='outer').fillna(0)
item_features = item_features.merge(item_order_count_features, on=['aid'], how='outer').fillna(0)
#item_features = item_features.drop_duplicates('aid')
#user_item_count = user_item_count.rename(columns={'session':'user','aid':'item'})
item_features

CPU times: user 17.6 s, sys: 951 ms, total: 18.5 s
Wall time: 18.4 s


,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features
aid,,,,,,
0,33,26,0.000000,33.0,0.0,0.0
1,32,29,0.031250,31.0,1.0,0.0
2,12,12,0.000000,12.0,0.0,0.0
3,1201,668,0.079933,1120.0,66.0,15.0
4,140,97,0.035714,135.0,5.0,0.0
...,...,...,...,...,...,...
1855598,6,6,0.000000,6.0,0.0,0.0
1855599,9,8,0.000000,9.0,0.0,0.0
1855600,68,42,0.073529,64.0,3.0,1.0


In [ ]:
del item_click_count_features, item_cart_count_features, item_order_count_features
gc.collect

<function gc.collect(generation=2)>

# step3 user特徴 : valid

In [ ]:
user_features = df_val.groupby('session').agg({'session': 'count', 'aid': 'nunique', 'type': 'mean'})
user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
#user_features.to_parquet('user_features.pqt')
user_features

,user_user_count,user_item_count,user_buy_ratio
session,,,
11098528,1,1,0.000000
11098529,1,1,0.000000
11098530,6,2,0.166667
11098531,24,11,0.333333
11098532,2,2,0.000000
...,...,...,...
12899774,1,1,0.000000
12899775,1,1,0.000000
12899776,1,1,0.000000


In [ ]:
#del 
#gc.collect

# step4 user × item 特徴量生成 : valid

このアイテムはユーザーによって既にクリック,cart,orderされていますか

In [ ]:
#user(session) x item(aid)に対するaction(type)の有無(1/0)を特徴量にする
def create_action_flag(df, type_number=0, column_name="user_item_click_flag"):
    action_flag_df = df.loc[df["type"] == type_number, ["session", "aid"]].drop_duplicates()
    action_flag_df[column_name] = 1
    return action_flag_df

In [ ]:
user_item_click_flag = create_action_flag(df_val, type_number=0, column_name="user_item_click_flag")
user_item_cart_flag = create_action_flag(df_val, type_number=1, column_name="user_item_cart_flag")
user_item_order_flag = create_action_flag(df_val, type_number=2, column_name="user_item_order_flag")

In [ ]:
tmp = pd.merge(
    user_item_click_flag,
    user_item_cart_flag,
    on=["session", "aid"],
    how="outer"
)
user_by_item_features = pd.merge(
    tmp,
    user_item_order_flag,
    on=["session", "aid"],
    how="outer"
).fillna(0)
user_by_item_features

,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag
0,11098528,11830,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0
2,11098530,264500,1.0,0.0,0.0
3,11098530,409236,1.0,1.0,0.0
4,11098531,452188,1.0,0.0,1.0
...,...,...,...,...,...
5535985,12896465,1551275,0.0,0.0,1.0
5535986,12896465,1306971,0.0,0.0,1.0
5535987,12896768,1303029,0.0,0.0,1.0
5535988,12898765,73333,0.0,0.0,1.0


In [ ]:
#df_val[df_val['type']==0].groupby('session').agg({'type': 'count'})

ユーザーはこの項目を複数回クリックしましたか? 幾つか

In [ ]:
#click
user_item_click_count = df_val[df_val['type']==0].groupby(['session','aid']).agg({'aid': 'count'})
user_item_click_count.columns = ['user_item_click_count']
user_item_click_count = df_val[['session','aid']].merge(user_item_click_count, how='left', on=['session','aid'])
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_item_click_count

#cart
user_item_cart_count = df_val[df_val['type']==1].groupby(['session','aid']).agg({'aid': 'count'})
user_item_cart_count.columns = ['user_item_cart_count']
user_item_cart_count = df_val[['session','aid']].merge(user_item_cart_count, how='left', on=['session','aid'])
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_item_cart_count

#order
user_item_order_count = df_val[df_val['type']==2].groupby(['session','aid']).agg({'aid': 'count'})
user_item_order_count.columns = ['user_item_order_count']
user_item_order_count = df_val[['session','aid']].merge(user_item_order_count, how='left', on=['session','aid'])
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_item_order_count

user_item_count = user_item_click_count.merge(user_item_cart_count, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.merge(user_item_order_count, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.drop_duplicates(['session','aid'])
#user_item_count = user_item_count.rename(columns={'session':'user','aid':'item'})

user_item_features = user_by_item_features.merge(user_item_count, on=['session','aid'], how='outer').fillna(0)
user_item_features

,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag,user_item_click_count,user_item_cart_count,user_item_order_count
0,11098528,11830,1.0,0.0,0.0,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0,1.0,0.0,0.0
2,11098530,264500,1.0,0.0,0.0,2.0,0.0,0.0
3,11098530,409236,1.0,1.0,0.0,3.0,1.0,0.0
4,11098531,452188,1.0,0.0,1.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...
5535985,12896465,1551275,0.0,0.0,1.0,0.0,0.0,1.0
5535986,12896465,1306971,0.0,0.0,1.0,0.0,0.0,1.0
5535987,12896768,1303029,0.0,0.0,1.0,0.0,0.0,1.0
5535988,12898765,73333,0.0,0.0,1.0,0.0,0.0,1.0


ユーザーがすでにクリック,cart,orderしたアイテムの数

In [ ]:
#click
user_per_item_click_count = df_val[df_val['type']==0].groupby('session').agg({'aid': 'count'})
user_per_item_click_count.columns = ['user_per_item_click_count']
user_per_item_click_count = df_val[['session','aid']].merge(user_per_item_click_count, how='left', on='session')
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_per_item_click_count

#cart
user_per_item_cart_count = df_val[df_val['type']==1].groupby('session').agg({'aid': 'count'})
user_per_item_cart_count.columns = ['user_per_item_cart_count']
user_per_item_cart_count = df_val[['session','aid']].merge(user_per_item_cart_count, how='left', on='session')
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_per_item_cart_count

#order
user_per_item_order_count = df_val[df_val['type']==2].groupby('session').agg({'aid': 'count'})
user_per_item_order_count.columns = ['user_per_item_order_count']
user_per_item_order_count = df_val[['session','aid']].merge(user_per_item_order_count, how='left', on='session')
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_per_item_order_count

user_item_count = user_per_item_click_count.merge(user_per_item_cart_count, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.merge(user_per_item_order_count, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.drop_duplicates(['session','aid'])
#user_item_count = user_item_count.rename(columns={'session':'user','aid':'item'})

user_item_features = user_item_features.merge(user_item_count, on=['session','aid'], how='outer').fillna(0)
user_item_features

,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag,user_item_click_count,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count
0,11098528,11830,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11098530,264500,1.0,0.0,0.0,2.0,0.0,0.0,5.0,1.0,0.0
3,11098530,409236,1.0,1.0,0.0,3.0,1.0,0.0,5.0,1.0,0.0
4,11098531,452188,1.0,0.0,1.0,2.0,0.0,1.0,20.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
5535985,12896465,1551275,0.0,0.0,1.0,0.0,0.0,1.0,5.0,3.0,9.0
5535986,12896465,1306971,0.0,0.0,1.0,0.0,0.0,1.0,5.0,3.0,9.0
5535987,12896768,1303029,0.0,0.0,1.0,0.0,0.0,1.0,11.0,3.0,2.0
5535988,12898765,73333,0.0,0.0,1.0,0.0,0.0,1.0,5.0,5.0,4.0


最後にクリック,cart,orderされたアイテムにflag

In [ ]:
'''
#実装あきらめた

#click
user_item_click_last = df_val[df_val['type']==0].groupby('session').agg({'aid': 'last'})
user_item_click_last.columns = ['user_item_click_last']
user_item_click_last = df_val[['session','aid']].merge(user_item_click_last, how='left', on='session')
#user_item_click_last.to_parquet('user_item_click_last.pqt')
user_item_click_last

#cart
user_item_cart_last = df_val[df_val['type']==1].groupby('session').agg({'aid': 'last'})
user_item_cart_last.columns = ['user_item_cart_last']
user_item_cart_last = df_val[['session','aid']].merge(user_item_cart_last, how='left', on='session')
#user_item_cuser_item_cart_lastlick_count.to_parquet('user_item_cart_last.pqt')
user_item_cart_last

#order
user_item_order_last = df_val[df_val['type']==2].groupby('session').agg({'aid': 'last'})
user_item_order_last.columns = ['user_item_order_last']
user_item_order_last = df_val[['session','aid']].merge(user_item_order_last, how='left', on='session')
#user_item_order_order.to_parquet('user_item_order_order.pqt')
user_item_order_last

user_item_count = user_item_click_last.merge(user_item_cart_last, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.merge(user_item_order_last, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.drop_duplicates(['session','aid'])
#user_item_count = user_item_count.rename(columns={'session':'user','aid':'item'})
user_item_count

user_item_features = user_item_features.merge(user_item_count, on=['session','aid'], how='outer').fillna(0)
user_item_features
'''

"\n#実装あきらめた\n\n#click\nuser_item_click_last = df_val[df_val['type']==0].groupby('session').agg({'aid': 'last'})\nuser_item_click_last.columns = ['user_item_click_last']\nuser_item_click_last = df_val[['session','aid']].merge(user_item_click_last, how='left', on='session')\n#user_item_click_last.to_parquet('user_item_click_last.pqt')\nuser_item_click_last\n\n#cart\nuser_item_cart_last = df_val[df_val['type']==1].groupby('session').agg({'aid': 'last'})\nuser_item_cart_last.columns = ['user_item_cart_last']\nuser_item_cart_last = df_val[['session','aid']].merge(user_item_cart_last, how='left', on='session')\n#user_item_cuser_item_cart_lastlick_count.to_parquet('user_item_cart_last.pqt')\nuser_item_cart_last\n\n#order\nuser_item_order_last = df_val[df_val['type']==2].groupby('session').agg({'aid': 'last'})\nuser_item_order_last.columns = ['user_item_order_last']\nuser_item_order_last = df_val[['session','aid']].merge(user_item_order_last, how='left', on='session')\n#user_item_order_order.t

In [ ]:
#del user_by_item_features, user_per_item_cart_count, user_per_item_order_count, user_item_click_last, user_item_cart_last, user_item_order_last, user_item_count
#gc.collect

In [ ]:
del user_by_item_features, user_per_item_cart_count, user_per_item_order_count, user_item_count
gc.collect

<function gc.collect(generation=2)>

#ラデック特徴量

In [ ]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 4.0 MB/s 


In [ ]:
#https://www.kaggle.com/code/radek1/polars-proof-of-concept-lgbm-ranker

import polars as pl

# to polars
df_val_pl = pl.DataFrame(df_val)

def add_action_num_reverse_chrono(df):
    return df.select([
        pl.col('*'),
        pl.col('session').cumcount().reverse().over('session').alias('action_num_reverse_chrono')
    ])

def add_session_length(df):
    return df.select([
        pl.col('*'),
        pl.col('session').count().over('session').alias('session_length')
    ])

def add_log_recency_score(df):
    linear_interpolation = 0.1 + ((1-0.1) / (df['session_length']-1)) * (df['session_length']-df['action_num_reverse_chrono']-1)
    return df.with_columns(pl.Series(2**linear_interpolation - 1).alias('log_recency_score')).fill_nan(1)

def add_type_weighted_log_recency_score(df):
    type_weights = {0:1, 1:6, 2:3}
    type_weighted_log_recency_score = pl.Series(df['log_recency_score'] / df['type'].apply(lambda x: type_weights[x]))
    return df.with_column(type_weighted_log_recency_score.alias('type_weighted_log_recency_score'))

def apply(df, pipeline):
    for f in pipeline:
        df = f(df)
    return df

#df_val
pipeline = [add_action_num_reverse_chrono, add_session_length, add_log_recency_score, add_type_weighted_log_recency_score]
radek_feature = apply(df_val_pl, pipeline)

radek_feature = radek_feature.to_pandas()
radek_feature = radek_feature.drop(['ts','type'], axis=1)

# step5 : step1に step2,3,4を追加

In [ ]:
#メモリ削減
candidates = reduce_mem_usage(candidates)
item_features = reduce_mem_usage(item_features)
user_features = reduce_mem_usage(user_features)
user_item_features = reduce_mem_usage(user_item_features)
radek_feature = reduce_mem_usage(radek_feature)

Memory usage of dataframe is 860.15 MB
Memory usage after optimization is: 472.65 MB
Decreased by 45.1%
Memory usage of dataframe is 97.49 MB
Memory usage after optimization is: 45.26 MB
Decreased by 53.6%
Memory usage of dataframe is 54.97 MB
Memory usage after optimization is: 24.05 MB
Decreased by 56.2%
Memory usage of dataframe is 464.60 MB
Memory usage after optimization is: 179.50 MB
Decreased by 61.4%
Memory usage of dataframe is 234.48 MB
Memory usage after optimization is: 117.24 MB
Decreased by 50.0%


In [ ]:
%%time
candidates = candidates.rename(columns={'session':'user'})

#step2
#item_features = pd.read_parquet('item_features.pqt')
candidates = candidates.merge(item_features, left_on='item', right_index=True, how='left').fillna(-1)

#step3
#user_features = pd.read_parquet('user_features.pqt')
user_features = user_features.rename(columns={'session':'user'})
candidates = candidates.merge(user_features, left_on='user', right_index=True, how='left').fillna(-1)

#step4
#user_features = pd.read_parquet('user_features.pqt')
user_item_features = user_item_features.rename(columns={'session':'user'})
user_item_features = user_item_features.rename(columns={'aid':'item'})
candidates = candidates.merge(user_item_features, on=['user','item'], how='left').fillna(0)
candidates

#radek
radek_feature = radek_feature.rename(columns={'session':'user'})
radek_feature = radek_feature.rename(columns={'aid':'item'})
candidates = candidates.merge(radek_feature, on=['user','item'], how='left').fillna(0)
candidates

CPU times: user 2min 9s, sys: 4.68 s, total: 2min 14s
Wall time: 2min 13s


,user,item,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features,user_user_count,user_item_count,...,user_item_click_count,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count,action_num_reverse_chrono,session_length,log_recency_score,type_weighted_log_recency_score
0,11098528,11830,32546.0,18362.0,0.175659,27910.0,3556.0,1081.0,1,1,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
1,11098528,588923,23956.0,14293.0,0.124939,21609.0,1701.0,646.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11098528,1732105,10121.0,5565.0,0.280762,7798.0,1804.0,519.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11098528,571762,18185.0,11918.0,0.125732,16435.0,1213.0,537.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11098528,884502,30419.0,17385.0,0.135742,27203.0,2304.0,913.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58513369,12899778,13568,855.0,527.0,0.173096,734.0,94.0,27.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58513370,12899778,1097813,462.0,328.0,0.151489,405.0,44.0,13.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58513371,12899778,1102546,1054.0,877.0,0.097717,974.0,57.0,23.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58513372,12899778,1068655,488.0,371.0,0.086060,452.0,30.0,6.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
del item_features, user_features, user_item_features, radek_feature
gc.collect

<function gc.collect(generation=2)>

# step6 : 学習できる形にするgt:0,gt:1

In [ ]:
#valid_labelをラベル、評価で分ける？
'''
valid_label = pd.read_parquet(INPUT_DIR+'test_labels.parquet')
from sklearn.model_selection import StratifiedKFold

type_labels = {'clicks':0, 'carts':1, 'orders':2}
valid_label['type'] = valid_label['type'].map(type_labels).astype('int8')

kf = StratifiedKFold(n_splits=2, shuffle=True)
for tr_idx, val_idx in kf.split(valid_label, valid_label['type']):
  tar = valid_label.loc[tr_idx, :].reset_index(drop=True)
  valid_eval = valid_label.loc[val_idx, :].reset_index(drop=True)

'''

"\nvalid_label = pd.read_parquet(INPUT_DIR+'test_labels.parquet')\nfrom sklearn.model_selection import StratifiedKFold\n\ntype_labels = {'clicks':0, 'carts':1, 'orders':2}\nvalid_label['type'] = valid_label['type'].map(type_labels).astype('int8')\n\nkf = StratifiedKFold(n_splits=2, shuffle=True)\nfor tr_idx, val_idx in kf.split(valid_label, valid_label['type']):\n  tar = valid_label.loc[tr_idx, :].reset_index(drop=True)\n  valid_eval = valid_label.loc[val_idx, :].reset_index(drop=True)\n\n"

In [ ]:
from traitlets.config.application import T
valid_labels = pd.read_parquet(INPUT_DIR+'test_labels.parquet')

#valid_labelsは評価の時に使うのでtar変数に入れる
tar = valid_labels.copy()
tar = tar.loc[valid_labels['type']==CFG.types]
tar['item'] = tar['ground_truth'].explode().astype('int32')
tar[CFG.types] = 1
tar = tar.rename(columns={'session':'user'})
tar = tar[['user', 'item', CFG.types]]
candidates = candidates.merge(tar,on=['user','item'],how='left').fillna(0)

candidates = reduce_mem_usage(candidates)
candidates

Memory usage of dataframe is 4799.03 MB
Memory usage after optimization is: 3682.98 MB
Decreased by 23.3%


,user,item,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features,user_user_count,user_item_count,...,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count,action_num_reverse_chrono,session_length,log_recency_score,type_weighted_log_recency_score,clicks
0,11098528,11830,32546.0,18368.0,0.175659,27910.0,3556.0,1081.0,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
1,11098528,588923,23956.0,14296.0,0.124939,21609.0,1701.0,646.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11098528,1732105,10121.0,5564.0,0.280762,7798.0,1804.0,519.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11098528,571762,18185.0,11920.0,0.125732,16435.0,1213.0,537.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11098528,884502,30419.0,17392.0,0.135742,27203.0,2304.0,913.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58513369,12899778,13568,855.0,527.0,0.173096,734.0,94.0,27.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58513370,12899778,1097813,462.0,328.0,0.151489,405.0,44.0,13.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58513371,12899778,1102546,1054.0,877.0,0.097717,974.0,57.0,23.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58513372,12899778,1068655,488.0,371.0,0.086060,452.0,30.0,6.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# stage2. ranker

# Training

In [ ]:
#スライスずれるのでtargetは最後のカラムにしておく必要あり。
FEATURES = list(candidates.columns[2:-1])
FEATURES

['item_item_count',
 'item_user_count',
 'item_by_count',
 'item_click_count_features',
 'item_cart_count_features',
 'item_order_count_features',
 'user_user_count',
 'user_item_count',
 'user_buy_ratio',
 'user_item_click_flag',
 'user_item_cart_flag',
 'user_item_order_flag',
 'user_item_click_count',
 'user_item_cart_count',
 'user_item_order_count',
 'user_per_item_click_count',
 'user_per_item_cart_count',
 'user_per_item_order_count',
 'action_num_reverse_chrono',
 'session_length',
 'log_recency_score',
 'type_weighted_log_recency_score']

In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#validを再度 train, valid　分割
from sklearn.model_selection import GroupKFold

kf = GroupKFold(n_splits=2)
for tr_idx, val_idx in kf.split(candidates, candidates[CFG.types], groups=candidates['user']):
  train = candidates.loc[tr_idx, :].reset_index(drop=True)
  valid = candidates.loc[val_idx, :].reset_index(drop=True)

In [ ]:
train

,user,item,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features,user_user_count,user_item_count,...,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count,action_num_reverse_chrono,session_length,log_recency_score,type_weighted_log_recency_score,clicks
0,11098528,11830,32546.0,18368.0,0.175659,27910.0,3556.0,1081.0,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
1,11098528,588923,23956.0,14296.0,0.124939,21609.0,1701.0,646.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11098528,1732105,10121.0,5564.0,0.280762,7798.0,1804.0,519.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11098528,571762,18185.0,11920.0,0.125732,16435.0,1213.0,537.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11098528,884502,30419.0,17392.0,0.135742,27203.0,2304.0,913.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29256682,12899778,13568,855.0,527.0,0.173096,734.0,94.0,27.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29256683,12899778,1097813,462.0,328.0,0.151489,405.0,44.0,13.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29256684,12899778,1102546,1054.0,877.0,0.097717,974.0,57.0,23.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29256685,12899778,1068655,488.0,371.0,0.086060,452.0,30.0,6.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
valid

,user,item,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features,user_user_count,user_item_count,...,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count,action_num_reverse_chrono,session_length,log_recency_score,type_weighted_log_recency_score,clicks
0,11098532,876469,5433.0,2922.0,0.057983,5159.0,233.0,41.0,2,2,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,1.000000,1.000000,0.0
1,11098532,7651,24997.0,15104.0,0.036804,24203.0,668.0,126.0,2,2,...,0.0,0.0,2.0,0.0,0.0,1.0,2.0,0.071777,0.071777,0.0
2,11098532,108125,88126.0,51552.0,0.040222,85064.0,2580.0,482.0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,11098532,1202618,9913.0,5684.0,0.030975,9634.0,251.0,28.0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,11098532,1159379,6426.0,3844.0,0.029419,6255.0,153.0,18.0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29256682,12899777,1361275,1572.0,925.0,0.089050,1450.0,104.0,18.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
29256683,12899777,1821821,857.0,492.0,0.092163,786.0,63.0,8.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
29256684,12899777,1544866,1538.0,775.0,0.096863,1412.0,103.0,23.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
29256685,12899777,1445372,1164.0,782.0,0.034363,1131.0,26.0,7.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


In [ ]:
from sklearn.model_selection import train_test_split

#時間で分けたいけど、とりあえずtrain_test_split
X_train = train[FEATURES]
y_train = train[CFG.types]
X_valid = valid[FEATURES]
y_valid = valid[CFG.types]

#参考　https://knuu.github.io/ltr_by_lightgbm.html
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg', #
    'lambdarank_truncation_level': 10,
    'ndcg_eval_at': 20,
    'n_estimators': 10,
    'boosting_type': 'dart',
    'random_state': 0,
}

lgb_train = lgb.Dataset(X_train, y_train, group=candidates.loc[X_train.index, :].groupby('user')['item'].agg('count').values ) #sessionごとに長さ異なること考慮(20ではない可能性あり)
#lgb_valid = lgb.Dataset(X_valid, y_valid, group=candidates.loc[X_valid.index, :].groupby('user')['item'].agg('count').values ) #sessionごとに長さ異なること考慮(20ではない可能性あり)
  
  
model = lgb.train(
    params, lgb_train, 
    #valid_sets=lgb_valid, 
    #early_stopping_rounds=20, #dartだと使えない
    verbose_eval=10  # 10 round毎に metric を表示
)

pickle.dump(model, open(OUTPUT_DIR + f'{CFG.exp}_{CFG.types}_lgb_model.pkl','wb'))

#del X_train, y_train, X_valid, y_valid, model
#gc.collect

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


# CV recall@k

In [ ]:
preds = model.predict(X_valid)
predictions = valid[['user','item']].copy()
predictions['pred'] = preds

predictions = predictions.sort_values(['user','pred'], ascending=[True,False]).reset_index(drop=True)
predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
predictions = predictions.loc[predictions.n<20]
pred_df = predictions.groupby('user').item.apply(list)
pred_df = pred_df.to_frame().reset_index()
pred_df.item = pred_df.item.apply(lambda x: " ".join(map(str,x)))
pred_df.columns = ['session','labels']
pred_df

,session,labels
0,11098532,7651 876469 108125 1202618 1159379 77906 17040...
1,11098533,385390 1165015 1074173 978918 1074173 1074173 ...
2,11098538,1550143 1263747 703265 717871 1711586 1711586 ...
3,11098545,1063103 14956 14956 1233788 14956 14956 14956 ...
4,11098546,152419 1541643 832949 832949 56629 623684 8329...
...,...,...
900621,12899770,277173 746786 1438555 737270 334172 1185386 91...
900622,12899772,1062165 878643 346692 398583 1741576 511000 17...
900623,12899774,33035 1539309 95488 743977 31490 1399483 10531...
900624,12899776,548599 1323160 135086 177902 899598 793373 114...


In [ ]:
#valid_labels も validを同じuserだけにする
valid_user = list(valid['user'].unique())
valid_labels = valid_labels.query('session==@valid_user').reset_index(drop=True)
valid_labels

,session,type,ground_truth
0,11098532,clicks,[1596491]
1,11098533,clicks,[1417450]
2,11098533,carts,"[108676, 1406660, 988295, 1118792, 1366413, 15..."
3,11098533,orders,"[935297, 652916, 334581, 1233050, 1366413, 118..."
4,11098538,clicks,[1452081]
...,...,...,...
1105602,12899770,clicks,[277173]
1105603,12899772,clicks,[1645259]
1105604,12899774,clicks,[1399483]
1105605,12899776,clicks,[1737908]


In [ ]:
%%time

#recall@k 計算関数
def recall_k_evaluate(test_labels, pred_df, session_types, top_k):
    sub = pred_df.copy()
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')][:top_k])
    test_labels = test_labels.loc[test_labels['type']==session_types]
    test_labels = test_labels.merge(sub, how='left', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1) 
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,top_k)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    print(f'{session_types} recall =',recall)

recall_k_evaluate(valid_labels, pred_df, CFG.types, CFG.recall)

clicks recall = 0.5143225280196132
CPU times: user 24.2 s, sys: 443 ms, total: 24.6 s
Wall time: 24.5 s


この後に、inferenceのために、candidates全体(lgb_train + lgb_valid)で再度lgb学習させる